In [1]:
import numpy as np

from params import mnistLip, cifarLip
from utils import params_to_dataframe

In [2]:
_, _, grid_params, _ = mnistLip()()
params_to_dataframe(grid_params, columns=['trn_acc', 'history'], result_file_dir='./results/experiment01/')

I1223 10:40:39.280750 140641119225664 auto_var.py:261] Running time: 0.002398
I1223 10:40:39.348590 140641119225664 auto_var.py:261] Running time: 0.002419
I1223 10:40:39.422923 140641119225664 auto_var.py:261] Running time: 0.002363
I1223 10:40:39.483220 140641119225664 auto_var.py:261] Running time: 0.002187
I1223 10:40:39.541884 140641119225664 auto_var.py:261] Running time: 0.002300
I1223 10:40:39.600006 140641119225664 auto_var.py:261] Running time: 0.002205


,attack,dataset,eps,model,norm,random_seed,trn_acc,history
0,pgd,mnist,1.0,ce-tor-CNN001,2,0,1.000000,"[{'epoch': 1, 'trn_loss': 0.44905163725217184,..."
1,pgd,mnist,1.0,tradesce-tor-CNN001,2,0,1.000000,"[{'epoch': 1, 'trn_loss': 0.48456251249313353,..."
2,pgd,mnist,1.0,advce-tor-CNN001,2,0,0.999667,"[{'epoch': 1, 'trn_loss': 0.7703302870114644, ..."
3,pgd,fashion,1.0,ce-tor-CNN001,2,0,0.999750,"[{'epoch': 1, 'trn_loss': 0.6706890512784323, ..."
4,pgd,fashion,1.0,tradesce-tor-CNN001,2,0,0.999617,"[{'epoch': 1, 'trn_loss': 0.663989037068685, '..."
5,pgd,fashion,1.0,advce-tor-CNN001,2,0,0.910800,"[{'epoch': 1, 'trn_loss': 1.0095622037887573, ..."


In [2]:
_, _, grid_params, _ = cifarLip()()
params_to_dataframe(grid_params, columns=['trn_acc', 'history'], result_file_dir='./results/experiment01/')

E1223 10:54:47.012559 139891478628160 auto_var.py:341] Error with {'attack': 'pgd', 'dataset': 'cifar10', 'eps': 10.0, 'model': 'ce-tor-WideResNet', 'norm': '2', 'random_seed': 0}
I1223 10:54:47.016836 139891478628160 auto_var.py:261] Running time: 0.002441
I1223 10:54:47.075744 139891478628160 auto_var.py:261] Running time: 0.000074


problem with ./results/experiment01/pgd-cifar10-10.0-ce-tor-WideResNet-2-0.pkl


,attack,dataset,eps,model,norm,random_seed,trn_acc,history
0,pgd,cifar10,10.0,tradesce-tor-WideResNet,2,0,0.78594,"[{'epoch': 1, 'trn_loss': 1.8665201709747314, ..."
